In [16]:
import advent
from scipy.ndimage import generic_filter

data = advent.get_char_grid(24, map_fn=lambda c: c == '#')

In [17]:
# part 1

def apply(x):
    if x[4] == 1: return (x[1] + x[3] + x[5] + x[7]) == 1
    elif x[4] == 0: return (x[1] + x[3] + x[5] + x[7]) in (1, 2)
    raise ValueError("?")

def biodiversity(x):
    return sum([2**i for i, v in enumerate(x.flatten()) if v])

hashes, h = set([]), None
new_board = data.copy()

while h not in hashes:
    hashes.add(h)
    new_board = generic_filter(new_board, apply, size=3, mode='constant', cval=False)
    h = hash(str(new_board.astype(int)))

print(biodiversity(new_board))

2129920


In [18]:
# Part 2

# Unfortunately we can't use generic filter anymore :(
# Adjacency rules: Any tile has 4 neighbors, except the ones neighboring the center, which have 3 + 5 = 8
from typing import NamedTuple

class Tile(NamedTuple):
    level: int
    row: int
    col: int
    
    def __str__(self):
        return f"{self.level},{self.row},{self.col}"

    def neighbors(self):
        if self.row == 2 and self.col == 2: raise ValueError("Huh?")
        if self.row == 0:
            yield Tile(self.level + 1, 1, 2)
        elif self.row == 4:
            yield Tile(self.level + 1, 3, 2)
        if self.col == 0:
            yield Tile(self.level + 1, 2, 1)
        elif self.col == 4:
            yield Tile(self.level + 1, 2, 3)
        if self.row == 1 and self.col == 2:
            for i in range(5):
                yield Tile(self.level - 1, 0, i)
        if self.row == 3 and self.col == 2:
            for i in range(5):
                yield Tile(self.level - 1, 4, i)
        if self.row == 2 and self.col == 1:
            for i in range(5):
                yield Tile(self.level - 1, i, 0)
        if self.row == 2 and self.col == 3:
            for i in range(5):
                yield Tile(self.level - 1, i, 4)
        for offset_x, offset_y in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
            row, col = self.row + offset_x, self.col + offset_y
            if row == 2 and col == 2:
                continue
            if row < 0 or row >= 5 or col < 0 or col >= 5:
                continue
            yield Tile(self.level, row, col)

print(str(Tile(0, 1, 2)))

0,1,2


In [ ]:
import itertools

# Since there are only 200 steps, and it takes 2 steps to go up a level, we can simulate all levels from -100 to 100
# This contains too many: it also contains the (x, 2, 2) which aren't really tiles so we'll skip them later
all_tiles = [Tile(level, row, col) for level, row, col in itertools.product(range(-100, 101), range(5), range(5))]
infested = set([])

for i in range(5):
    for j in range(5):
        if data[i, j]: infested.add(Tile(0, i, j))

def step(infested):
    new_infested = set([])
    for tile in all_tiles:
        if tile.row == 2 and tile.col == 2: continue
        neighbors = sum([1 for neighbor in tile.neighbors() if neighbor in infested])
        if tile in infested:
            if neighbors == 1:
                new_infested.add(tile)
        else:
            if neighbors in (1, 2):
                new_infested.add(tile)
    return new_infested

for _ in range(200):
    infested = step(infested)

print(len(infested))

99
